# Base functions
> Functions to complete the RAG process.

## Verify if exist all the necessary files

In [1]:
# %%script false --no-raise-error
from pathlib import Path
import json

# Define the relative path to the 'data' directory and the document name
data_dir = Path('..', 'data')
# Create the 'data' directory if it doesn't exist
data_dir.mkdir(parents=True, exist_ok=True)

document_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json"

document_name = "documents.json"
document_path = Path(data_dir, document_name)

if not document_path.is_file():
    !wget {document_url} -O {document_path}

with open(document_path, 'rt') as f_in:
    documents_raw = json.load(f_in)

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


file_path = "minsearch.py"
path = Path(file_path)

if not path.is_file():
    !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

## If we want to download the base document file instead to use the saved in disk

In [2]:
%%script false --no-raise-error

import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Search index using TF-IDF and cosine similarity for text fields and exact matching for keyword fields.


In [3]:
from minsearch import Index

index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
query = "Can I join the course if it has already started?"

filter_dict = {"course": "data-engineering-zoomcamp"}
boost_dict = {"question": 3, "section": 0.5}
minsearch_results = index.search(query, 
                       filter_dict, 
                       boost_dict, 
                       num_results=3)

for result in minsearch_results:
    print(result)

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}
{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}
{'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check t

In [5]:
def minsearch_index_search(query: str, filter_dict: dict={}, boost_dict: dict={}, num_results: int=3):
    results = index.search(query, filter_dict, boost_dict, num_results)
    return results  

In [6]:
query = "Can I join the course if it has already started?"
filter_dict = {"course": "data-engineering-zoomcamp"}
boost_dict = {"question": 3, "section": 0.5}

minsearch_results = minsearch_index_search(query, filter_dict, boost_dict)

for result in minsearch_results:
    print(result)

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}
{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}
{'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check t

## Create Vector storage (ElasticSearch)

- How to create a docker container for ElasticSearch:

```bash
docker run -d \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

- How to verify if it is running, execute in a `terminal`:

```bash
curl http://localhost:9200
```

- This is the index setting:  

```json
{
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
```

- This is a query example:  

```json
{
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
```

In [7]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "homework-course"
# Check if the index exists
if es_client.indices.exists(index=index_name):
    print(f"The index '{index_name}' already exists.")
else:
    es_client.indices.create(index=index_name, body=index_settings)
    for doc in tqdm(documents):
        es_client.index(index=index_name, document=doc)
    
    print(f"The index '{index_name}' has been created.")


The index 'homework-course' already exists.


In [8]:
def elastic_search(index_name: str, query: str, fields: list[str], filter_course: str, num_results: int=5):
    search_query = {
        "size": num_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": fields,
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": filter_course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return result_docs

In [9]:
%%script false --no-raise-error

index_name = "course-questions"
query = "I just discovered the course. Can I still join it?"
fields = ["question^3", "text", "section"]
filter_course="data-engineering-zoomcamp"
elastic_search(index_name, query, fields, filter_course)

## Create the prompt, LLM and RAG functions

In [10]:
def build_user_prompt(query: str, search_result: str):
    user_prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
S: {section}
Q: {question}
A: {text}
""".strip()

    context= ""
    
    for doc in search_result:
        context += f"{context_template.format(section=doc['section'], question=doc['question'], text=doc['text'])}\n\n"

    user_prompt = user_prompt_template.format(question=query, context=context).strip()
    return user_prompt

In [11]:
index_name = "course-questions"
query="How do I execute a command in a running docker container?"
fields = ["question^4", "text", "section"]
filter_course="machine-learning-zoomcamp"
search_result = elastic_search(index_name, query, fields, filter_course, num_results=3)
prompt = build_user_prompt(query, search_result)

In [12]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
S: 5. Deploying Machine Learning Models
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

S: 5. Deploying Machine Learning Models
Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basi

In [13]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

In [14]:
def llm(user_prompt: str, model: str="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages= [
            {"role": "user", "content": user_prompt},
        ]
    )
    return response.choices[0].message.content

In [15]:
def rag_elastic(index_name: str, query: str, fields: list[str], filter_course: str, num_results: int, model ):
    search_result = elastic_search(index_name, query, fields, filter_course, num_results)
    user_prompt = build_user_prompt(query, search_result)
    answer = llm(user_prompt, model)
    return answer


In [17]:
index_name = "course-questions"
query="How do I execute a command in a running docker container?"
fields = ["question^4", "text", "section"]
filter_course="machine-learning-zoomcamp"
num_results = 3
model = "gpt-3.5-turbo"
answer = rag_elastic(index_name, query, fields, filter_course, num_results, model)
answer

'To execute a command in a running docker container, you need to first find the container ID by running `docker ps`, then use the `docker exec -it <container-id> bash` command to execute a specific command in that container.'